---

<center>

# **Uncertainty Quantification**

<center>

---


## Dynamic Problem:

---

Now that we've tackled the [static problem](03_Static_Problem.ipynb) were we identified an uncertain parameter ($\hat{k}$) now we are going to dive into **uncertainty propagation and identification in a dynamical model**.

Consider a mechanical system represented by Fig 4.1, suppose now that given the operational conditions and the characteristics of this system we can safely assume that the situation can be modeled by a **discrete 1 DOF** (*degree of freedom*) model. Thus the equation of motion is given by

\begin{equation}
\tag{4.01}
m\ddot{x}(t)+c\dot{x}+kx(t)=f(t)
\end{equation}

where

* $m$: mass of the system
* $c$: damping coefficient
* $k$: stiffness of the system

<center> <img src="diagrams\04_dynamical_model.svg" width="500" height="400"> 

*Fig 4.01: mechanical dynamical system*

</center>

Now we are going to perform behavioural analysis of the system in two domains; **frequency** and **time**

### <u>4.1 Frequency domain analysis</u>:

The following analysis will be made using the [Frequency Response Function](note_frequency_response_function.ipynb) (*FRF*) of the system.

Applying Fourier transform to Eq. 4.0.1 we get

\begin{equation}
\tag{4.1.1}
(-m\omega^2+jc\omega+k)\tilde{x}(\omega)=\tilde{f}(\omega)
\end{equation}

where 

* $j=\sqrt{-1}$: is the imaginary unit
* $\tilde{x}(\omega)$: Fourier transform of $x(t)$
* $\tilde{f}(\omega)$: Fourier transform of $f(t)$

Now,  we obtain the FRF of the system $h(\omega)$ as the ratio between $\tilde{x}(\omega)$ and $\tilde{f}(\omega)$, that is

\begin{equation}
\tag{4.1.2}
h(\omega)=\frac{\tilde{x}(\omega)}{\tilde{f}(\omega)}=(-\omega^2m+\omega cj+k)^{-1}=\frac{1}{m}\left[(\omega_n^2-\omega^2)+2\zeta\omega_n\omega j\right]^{-1}
\end{equation}

The symbol $w_n\sqrt{k/m}$ is the **natural frequency** of the undamped system and $\zeta=c/(2m\omega_n)$ is the **damping ratio**.

First of all **we are going to analyze the effect of the uncertainty in $\zeta$ on the system response**. In this manner, $\zeta$ is going to be represented by a random variable, $\Xi$. As a consequence of this, the displacement of the system for a instant $t=t_s$ is a random variable $X(t_s,\zeta)$ and by extension, for each frequency $\omega=\omega_r$ we have $H(\omega_r,\zeta)$.

The random variable $H(\omega_r,\zeta)$ has a FRF in general form

\begin{equation}
\tag{4.1.3}
H(\omega,\Xi)=\frac{\tilde{X}(\omega,\Xi)}{\tilde{f}(\omega)}=\frac{1}{m}\left[(\omega_n^2-\omega^2)+2\Xi\omega_n\omega j\right]^{-1}
\end{equation}

Equation 4.1.3 is a complete probabilistic model of the system in so far we need to know the value of the uncertain parameter $\Xi$ to obtain information about $H(\omega_r)$

#### 4.1.1 Python Implementation

Now that we have the probabilistic model set it is time to tackle the problem and solve it.

First of all we are going to suppose that the uncertain parameter $\Xi$ follows an uniform probability distribution with minimum $\zeta=0.05$ and maximum $\zeta=0.20$, that is $\Xi \sim U(0.05,0.2)$.

The next thing is start writing the code, first we are going to build a function that solves for the FRF given the mass `m`, damping coefficient `c`, stiffness `k` and the frequency interval array `freq`.

In [1]:
import numpy as np

def solve_FRF(m, c, k, freq):
    """
    Calculate the Frequency Response Function (FRF) for a 1 DOF system.
    
    Parameters:
    -----------
    m : float
        Mass of the system
    c : float
        Damping coefficient
    k : float
        Stiffness coefficient
    freq : array-like
        Array of frequencies to calculate the FRF
        
    Returns:
    --------
    numpy.ndarray
        Magnitude of the FRF at each frequency
    """
    H = 1/(-m * (freq**2) + c*1j*freq + k)
    return np.abs(H)

Now the messy part, first we import the necessary libraries and then define some basics parameters of the system

In [2]:
import numpy as np
from numpy.random import uniform
import matplotlib.pyplot as plt

# Number of Monte Carlo simulations
n_sMC = 41

# Number of grid points
grid_points = 1001

# Mass
m = 1

# Fixed natural frequency
wn = 8

# Frequency array
w_min = 0
w_max = 15
freq = np.linspace(w_min, w_max, grid_points)

# Parameter uncertainty ranges
ximin = 0.05
ximax = 0.2

Now we'll make the `loop` that is going to perform the Monte Carlo simulation

In [3]:
# Allocate empty matrix for the simulation
# Row --> frequency values
# Column --> Monte Carlo simulations
results_frf = np.zeros((grid_points, n_sMC))

for iMC in range(n_sMC):

    xi = uniform(ximin, ximax)                              # Xi ~ U(0.05, 0,2)

    wn = 8                                                  # Fixed natural frequency
    c = 2 * xi * wn * m                                     # Damping coefficient
    k = (wn**2) * m                                         # Stiffness

    results_frf[:, iMC] = solve_FRF(m, c, k, freq)  


In [ ]:
# Calculate FRF with minimum damping ratio
c = 2 * ximin * wn * m
k = wn**2 * m
min_frf = solve_FRF(m, c, k, freq)

# Calculate FRF with maximum damping ratio
c = 2 * ximax * wn * m
k = wn**2 * m
max_frf = solve_FRF(m, c, k, freq)

plt.ioff()
fig, axis = plt.subplots(figsize=(11, 7))
plt.plot(freq, min_frf, 'r', linewidth=4, label=rf'$\zeta_{{min}}={ximin}$')
plt.plot(freq, max_frf, 'g', linewidth=4, label=rf'$\zeta_{{max}}={ximax}$')
plt.plot(freq, results_frf, 'k', linewidth=0.75)
plt.plot([], [], 'k', linewidth=0.75, label=r'$0.05 < \zeta < 0.2$')
axis.set_xlabel(r'$\omega$ [rad/s]', fontsize=14)
axis.set_ylabel(r'$|H| [m/N]$', fontsize=14)
axis.grid(linestyle='--', alpha=0.7)
axis.legend(fontsize=12)
axis.set_ylim(0, 0.17)
axis.set_xlim(0, 15)
axis.set_title('Frequency Response Function with Parameter Uncertainty', fontsize=18, fontweight='bold', pad=10)
plt.savefig('diagrams/04_frf_uncertain_zeta.svg', bbox_inches='tight', pad_inches=0.4, dpi=300)

The generated figure is presented in Fig. 4.1.1 where 40 Monte Carlo simulations where made for the FRF $H(\omega,\Xi)$. We clearly can see drawn in red an green the upper and lower limits, respectively, for the values of $\zeta$. Also in black are drawn each of the MC simulations.

<center> <img src="diagrams\04_frf_uncertain_zeta.svg" width="700" height="500"> 

*Fig 4.1.1: frequency response for uncertain damping ratio*

</center>

In [70]:
# Calculate percentiles for the probability envelope
lim_sup = np.percentile(results_frf, 99, axis=1)
lim_inf = np.percentile(results_frf, 1, axis=1)

# Calculate mean response
mean_response = np.mean(results_frf, axis=1)

# Plot mean and probability envelope
plt.ioff()
fig, axis = plt.subplots(figsize=(11, 7))
plt.plot(freq, lim_sup, color='#3a0ca3', linewidth=3, label='98% probability envelope')
plt.plot(freq, lim_inf, color='#3a0ca3', linewidth=3)
plt.plot(freq, mean_response, '--g', linewidth=3, label='Mean response')
axis.set_xlabel(r'$\omega$ [rad/s]', fontsize=14)
axis.set_ylabel(r'$|H| [m/N]$', fontsize=14)
axis.set_ylim(0, 0.17)
axis.set_xlim(0, 15)
axis.legend(fontsize=12)
axis.grid(linestyle='--', alpha=0.7)
axis.set_title('Mean Response and Probability Envelope', fontsize=18, fontweight='bold', pad=10)
plt.savefig('diagrams/04_frf_envelope.svg', bbox_inches='tight', pad_inches=0.4, dpi=300)

Here (Fig. 4.1.2) we have a more pleasing way to show the results of our analysis, the blue lines represent a confiability interval (envelope) where we are 98% sure that the FRF of the system is going to be given the uncertainty in $\zeta$. The dotted green line represents the mean frequency response.

<center> <img src="diagrams\04_frf_envelope.svg" width="700" height="500"> 

*Fig 4.1.2: 98% envelope for the FRF*

</center>

An engineering team may make a very important conclusion from Fig. 4.1.1 or 4.1.2, for a range of frequencies $\omega = [0, 6] \frac{rad}{s}$ the system shows a quasi-static behaviour as the frequency response is very little.

#### 4.1.2 Parameter identification

So far we have been finding how the system behaves for a given distribution of the uncertain parameter based solely in a computational model. Now we are going to identify the uncertain parameter as we did in [notebook 3](03_Static_Problem.ipynb) given a set of measurements of the real frequency response of the system.

Our focus remain with the damping ratio $\zeta$, now suppose we have experimental data {$H^{exp}(\omega_1), H^{exp}(\omega_2), ..., H^{exp}(\omega_N)$} and the frequency range that the measurement device can capture {$\omega_1, \omega_2, ..., \omega_N$}.

The data related to the FRF measurement of the test device will be generated following 

\begin{equation}
\tag{4.1.4}
H^{exp}(\omega_r) = H(\zeta, \omega_r)(1+0.15\nu_r)
\end{equation}

where $\nu_r \sim N(0, 1)$.

Now, the identification of $\zeta$ is going to be formulated as a optimization problem given

\begin{equation}
\tag{4.1.5}
\hat{\zeta} = argmin_{\hspace{0.25mm}\zeta\hspace{0.3mm}\in\hspace{0.3mm}\mathcal{D}} \sum_{r=1}^{N}(|H^{exp}(\omega_r)|-|H(\zeta, \omega_r)|)^2 
\end{equation}

where $\mathcal{D}$ is the feasible search set for the parameter $\zeta$.

The minimization problem can be tackled using various methods, in particular we are going to use the [Levenberg-Marquardt method](note_levenberg-marquardt.ipynb).

#### 4.1.4 Python implementation

The Levenberg-Marquardt method is implemented using `Scipy`s library called `ompitimze`, in specific using the function `scipy.optimize.least_squares` wich can be tuned to use various methods, including the one already mentioned

Firts, import libraries, define general data and system parameters

In [61]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from numpy.random import randn

# Number of experimental data and reference points
n_ref = 101
n_exp = 101

# Number of grid points
grid_points = 1001

# Mass
m = 1

# True natural frequency
wn_true = 8

# True damping ratio
zeta_true = 0.125

# System parameters
c = 2*zeta_true*wn_true*m
k = (wn_true**2)*m

# Frequency array
w_min = 0
w_max = 15
freq = np.linspace(w_min, w_max, grid_points)


Now, hands on the dough

In [77]:
# True frequency response for reference
true_frf = solve_FRF(m, c, k, freq)

# Creating experimental data
freq_exp = np.linspace(w_min, w_max, n_exp)
aux_frf = solve_FRF(m, c, k, freq_exp)
exp_frf = aux_frf * (1 + 0.15 * randn(len(aux_frf)))        # 15% standard deviation normal noise

plt.ioff()
fig, axis = plt.subplots(figsize=(11, 7))
plt.scatter(freq_exp, exp_frf, color='red', label='Measurements', facecolor='none', edgecolor='red', linewidth=2)
plt.plot(freq, true_frf, linewidth=1.75, linestyle='--',color='blue', label=fr'True FRF ($\zeta = {zeta_true})$')
axis.set_xlabel(r'$\omega$ [rad/s]', fontsize=14)
axis.set_ylabel(r'$|H| [m/N]$', fontsize=14)
axis.set_ylim(0, 0.08)
axis.set_xlim(0, 15)
axis.set_title('True FRF vs Experimental data', fontsize=18, fontweight='bold', pad=10)
axis.legend(fontsize=12)
axis.grid(linestyle='--', alpha=0.7)
# plt.savefig('diagrams/04_frf_exp_data.svg', bbox_inches='tight', pad_inches=0.4, dpi=300)

First we contrast the reference (true FRF) vs the experimental, measured, data. 

This can be seen in Fig. 4.1.3

<center> <img src="diagrams\04_frf_exp_data.svg" width="700" height="500"> 

*Fig 4.1.3: experimental data vs true reference FRF*

</center>

Then we solve the optimization problem and plot the results. For that purpose first we define a function that returns the error that we have to minimize, that is the argument of our optimization problem.

The function that we are going to use, `scipy.optimize.least_squares` needs a function that returns an array of residuals, that is, the error at each individual data point. Then the optimizer minimizes the sum of squared residuals.

In [63]:
def get_residuals(zeta, *args):
    """
    Calculates the error between the reference and the identified model.
    
    Parameters:
    -----------
    exp_frf : numpy.darray
        Measurements of the FRF
    wn_true : float
        True natural frequency
    freq_exp : numpy.darray
        Frequency measurements associated with exp_frf
    m : float
        Mass
    k : float
        Stiffness of the system 
    zeta : float
        Initial guess for damping coefficient        
    Returns:
    --------
    residuals : numpt.darray
        Array with the point-by-point differences between model and data
    """
    
    exp_frf, wn_true, freq_exp, m, k = args

    c = 2 * zeta * wn_true * m

    aux_frf = solve_FRF(m, c, k, freq_exp)
    residuals = aux_frf - exp_frf

    return residuals

In [ ]:
# Initial guess for damping ratio
zeta_i0 = 0.05

# Identified damping ratio
zeta_id = least_squares(
    get_residuals,
    zeta_i0,
    args=(exp_frf, wn_true, freq_exp, m, k),
    method='lm'
).x[0]

# Identified model
c_id = 2 * wn_true * zeta_id * m
id_frf = solve_FRF(m, c_id, k, freq)

# Plot
plt.ioff()
fig, axis = plt.subplots(figsize=(11, 7))
plt.scatter(freq_exp, exp_frf, color='red', label='Measurements', facecolor='none', edgecolor='red', linewidth=1.5)
plt.plot(freq, true_frf, linewidth=1.5, linestyle='--',color='blue', label=fr'True FRF ($\zeta = {zeta_true})$')
plt.plot(freq, id_frf, linewidth=2, linestyle='-',color='magenta', label=fr'Identified FRF ($\hat{{\zeta}} = {zeta_id:.3f})$')
axis.set_xlabel(r'$\omega$ [rad/s]', fontsize=14)
axis.set_ylabel(r'$|H| [m/N]$', fontsize=14)
axis.set_ylim(0, 0.08)
axis.set_xlim(0, 15)
axis.set_title(r'Identified model with $\hat{\zeta}$', fontsize=18, fontweight='bold', pad=10)
axis.legend(fontsize=12)
axis.grid(linestyle='--', alpha=0.7)
# plt.savefig('diagrams/04_identified_frf.svg', bbox_inches='tight', pad_inches=0.4, dpi=300)

Now we have succesfully identified the uncertain damping ratio, Fig. 4.1.4 shows in magenta the FRF of the identified model

<center> <img src="diagrams\04_identified_frf.svg" width="700" height="500"> 

*Fig 4.1.4: reference vs identified model frf*

</center>

In [80]:
# Initial guess for damping ratio
zeta_i0 = 0.05

# Identified damping ratio
zeta_id = least_squares(
    get_residuals,
    zeta_i0,
    args=(exp_frf, wn_true, freq_exp, m, k),
    method='lm'
).x[0]

# Identified model
c_id = 2 * wn_true * zeta_id * m
id_frf = solve_FRF(m, c_id, k, freq)

# Plot
plt.ioff()
fig, axis = plt.subplots(2, 1, figsize=(11, 7), sharex=True)
axis[0].scatter(freq_exp, exp_frf, color='red', label='Measurements', facecolor='none', edgecolor='red', linewidth=1.5)
axis[0].plot(freq, true_frf, linewidth=1.5, linestyle='--',color='blue', label=fr'True FRF ($\zeta = {zeta_true})$')
axis[0].plot(freq, id_frf, linewidth=2, linestyle='-',color='magenta', label=fr'Identified FRF ($\hat{{\zeta}} = {zeta_id:.3f})$')
axis[0].set_ylabel(r'$|H| [m/N]$', fontsize=12)
axis[0].set_ylim(0, 0.08)
axis[0].set_xlim(0, 15)
axis[0].set_title(r'Identified model with $\hat{\zeta}$', fontsize=14, fontweight='bold', pad=10)
axis[0].legend()
axis[0].grid(linestyle='--', alpha=0.7)

residuals = id_frf - true_frf
axis[1].plot(freq, residuals, color='green', label='Residuals', linewidth=2)
axis[1].set_xlabel(r'$\omega$ [rad/s]', fontsize=12)
axis[1].set_title('Residuals between Identified and True Models', fontweight='bold')
axis[1].set_xlim(0, 15)
axis[1].grid(linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('diagrams/04_identified_frf_residuals.svg', bbox_inches='tight', pad_inches=0.4, dpi=300)

The error between the identified model and the true model is very small relative to the small data set that we used, this can be appreciated in Fig. 4.1.4 or Fig 4.1.5 where the residuals were plotted with the identified model in the same figure.

<center> <img src="diagrams\04_identified_frf_residuals.svg" width="700" height="500"> 

*Fig 4.1.5: residuals between identified and true model*

</center>